In [1]:
using Agents

## Las tres cosas MÁS importantes de un modelo
* La definición del espacio (Grafo, rejilla, etc).
* La definición de los tipos de agentes.
* La definición de las reglas de evolución. Evolución discreta o continua.

### Definición del espacio: En este caso Grid)

### Definición de los tipos de agentes: En este caso solo un tipo de agente)

In [2]:
@agent struct GoLAgent(GridAgent{2})
    status::Bool # Si el agente está vivo o muerto
end

### Definición de las reglas de evolución o pasos.
En GoL las reglas son:
* Cualquier célula viva que que tenga a su alrededor (8 casillas) 2 o 3 células vivas, sigue viva. Si tiene más muere de superpoblación y si tiene menos muere de soledad. 

* Si una célula muerta tiene 3 células vivas alrededor, revive. 

In [3]:
function gol_step!(agent, model)
    minlive = model.min_to_live
    maxlive = model.max_to_live
    count_neighbors_alive = 0

    # for each neighbor, get the status and compare
    # increment 'count_neighbors_alive' as appropriately.
    for neigh in nearby_agents(agent, model)
        count_neighbors_alive += neigh.status
    end

    if agent.status == true
        if count_neighbors_alive >= minlive && count_neighbors_alive <= maxlive
            agent.status = true
        else
            agent.status = false
        end
    else
        if count_neighbors_alive == maxlive
            agent.status = true
        end
    end
    return
end

gol_step! (generic function with 1 method)

### Definición del espacio y otras propiedades
Función de inicializar y otras propiedades (min to live, max to live y la seed)

In [4]:
function initialize(; total_agents, gridsize, min_to_live, max_to_live, seed)
    space = GridSpaceSingle(gridsize; periodic = true)
    properties = Dict(:min_to_live => min_to_live, :max_to_live => max_to_live)
    rng = Xoshiro(seed)
    model = StandardABM(
        GoLAgent, space;
        agent_step! = gol_step!, properties, rng,
        container = Vector, # agents are not removed, so we us this
        scheduler = Schedulers.Randomly() # all agents are activated once at random
    )
    # populate the model with agents, adding equal amount of the two types of agents
    # at random positions in the model. At the start all agents are unhappy.
    for n in 1:total_agents
        add_agent_single!(model; status = true)
    end
    return model
end

initialize (generic function with 1 method)

## Representación

In [5]:
using CairoMakie
using Random: Xoshiro

In [7]:
groupcolor(a) = a.status == false ? :white : :black
g_size = [50, 50]
p_live_agents = 0.30
nb_total_agents = trunc(Int, (g_size[1] * g_size[2]) * p_live_agents)
nb_total_agents = nb_total_agents

println(nb_total_agents)

game_of_life = initialize(total_agents = nb_total_agents, gridsize = Tuple(g_size), min_to_live = 2, max_to_live = 3, seed = 125)


750


StandardABM with 750 agents of type GoLAgent
 agents container: Vector
 space: GridSpaceSingle with size (50, 50), metric=chebyshev, periodic=true
 scheduler: Agents.Schedulers.Randomly
 properties: min_to_live, max_to_live

In [18]:
size(abmspace(game_of_life))

(50, 50)

In [13]:
abmproperties(game_of_life)

Dict{Symbol, Int64} with 2 entries:
  :min_to_live => 2
  :max_to_live => 3

In [6]:
abmvideo(
    "gol.mp4", game_of_life;
    agent_color = groupcolor, agent_marker = :rect, as = 13,
    framerate = 9, frames = 20,
    title = "Game Of Life Simulation"
)

LoadError: UndefVarError: `game_of_life` not defined in `Main`
Suggestion: check for spelling errors or missing imports.